In [1]:
!pip install flaml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from lightgbm import LGBMRegressor
import lightgbm as lgb
from sklearn.preprocessing import PowerTransformer

In [4]:
#train/test/submission dataset load
train_data = pd.read_csv('/content/drive/MyDrive/데이콘/따릉이/train.csv')
test_data = pd.read_csv('/content/drive/MyDrive/데이콘/따릉이/test.csv')
submission_data = pd.read_csv('/content/drive/MyDrive/데이콘/따릉이/submission.csv')

# **1. Train/Test Split**


In [5]:
train_data.head()

,id,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5,count
0,3,20,16.3,1.0,1.5,89.0,576.0,0.027,76.0,33.0,49.0
1,6,13,20.1,0.0,1.4,48.0,916.0,0.042,73.0,40.0,159.0
2,7,6,13.9,0.0,0.7,79.0,1382.0,0.033,32.0,19.0,26.0
3,8,23,8.1,0.0,2.7,54.0,946.0,0.040,75.0,64.0,57.0
4,9,18,29.5,0.0,4.8,7.0,2000.0,0.057,27.0,11.0,431.0


In [6]:
train_data.describe()

,id,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5,count
count,1459.000000,1459.000000,1457.000000,1457.000000,1450.000000,1457.000000,1457.000000,1383.000000,1369.000000,1342.000000,1459.000000
mean,1105.914325,11.493489,16.717433,0.031572,2.479034,52.231297,1405.216884,0.039149,57.168736,30.327124,108.563400
std,631.338681,6.922790,5.239150,0.174917,1.378265,20.370387,583.131708,0.019509,31.771019,14.713252,82.631733
min,3.000000,0.000000,3.100000,0.000000,0.000000,7.000000,78.000000,0.003000,9.000000,8.000000,1.000000
25%,555.500000,5.500000,12.800000,0.000000,1.400000,36.000000,879.000000,0.025500,36.000000,20.000000,37.000000
50%,1115.000000,11.000000,16.600000,0.000000,2.300000,51.000000,1577.000000,0.039000,51.000000,26.000000,96.000000
75%,1651.000000,17.500000,20.100000,0.000000,3.400000,69.000000,1994.000000,0.052000,69.000000,37.000000,150.000000
max,2179.000000,23.000000,30.000000,1.000000,8.000000,99.000000,2000.000000,0.125000,269.000000,90.000000,431.000000


In [7]:
test_data.head()

,id,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5
0,0,7,20.7,0.0,1.3,62.0,954.0,0.041,44.0,27.0
1,1,17,30.0,0.0,5.4,33.0,1590.0,0.061,49.0,36.0
2,2,13,19.0,1.0,2.1,95.0,193.0,0.020,36.0,28.0
3,4,6,22.5,0.0,2.5,60.0,1185.0,0.027,52.0,38.0
4,5,22,14.6,1.0,3.4,93.0,218.0,0.041,18.0,15.0


In [8]:
test_data.describe()

,id,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5
count,715.000000,715.000000,714.000000,714.000000,714.000000,714.000000,714.000000,680.000000,678.000000,679.000000
mean,1056.854545,11.472727,23.263305,0.051821,2.388515,56.668067,1359.494398,0.041196,36.930678,24.939617
std,625.574471,6.928427,4.039645,0.221820,1.171270,19.661880,614.171870,0.020656,12.641503,10.075857
min,0.000000,0.000000,14.600000,0.000000,0.000000,16.000000,126.000000,0.003000,9.000000,7.000000
25%,507.000000,5.500000,20.300000,0.000000,1.500000,42.000000,807.250000,0.027000,28.000000,17.000000
50%,1046.000000,11.000000,22.900000,0.000000,2.300000,55.000000,1489.000000,0.039000,35.000000,24.000000
75%,1616.500000,17.000000,26.375000,0.000000,3.275000,73.750000,2000.000000,0.052250,45.000000,31.000000
max,2177.000000,23.000000,33.800000,1.000000,5.900000,100.000000,2000.000000,0.138000,94.000000,69.000000


In [9]:
#결측치 대체
#평균값으로 대체함.
train_data.isna().sum()

id                          0
hour                        0
hour_bef_temperature        2
hour_bef_precipitation      2
hour_bef_windspeed          9
hour_bef_humidity           2
hour_bef_visibility         2
hour_bef_ozone             76
hour_bef_pm10              90
hour_bef_pm2.5            117
count                       0
dtype: int64

In [10]:
#결측치 대체
#평균값으로 대체함.
test_data.isna().sum()

id                         0
hour                       0
hour_bef_temperature       1
hour_bef_precipitation     1
hour_bef_windspeed         1
hour_bef_humidity          1
hour_bef_visibility        1
hour_bef_ozone            35
hour_bef_pm10             37
hour_bef_pm2.5            36
dtype: int64

In [11]:
train_data['hour_bef_temperature'].fillna(value = train_data['hour_bef_temperature'].sum() / (len(train_data) - len(str(train_data['hour_bef_temperature'].isna().sum()))), inplace=True)
train_data['hour_bef_precipitation'].fillna(value = train_data['hour_bef_precipitation'].sum() / (len(train_data) - len(str(train_data['hour_bef_precipitation'].isna().sum()))), inplace=True)
train_data['hour_bef_windspeed'].fillna(value = train_data['hour_bef_windspeed'].sum() / (len(train_data) - len(str(train_data['hour_bef_windspeed'].isna().sum()))), inplace=True)
train_data['hour_bef_humidity'].fillna(value = train_data['hour_bef_humidity'].sum() / (len(train_data) - len(str(train_data['hour_bef_humidity'].isna().sum()))), inplace=True)
train_data['hour_bef_visibility'].fillna(value = train_data['hour_bef_visibility'].sum() / (len(train_data) - len(str(train_data['hour_bef_visibility'].isna().sum()))), inplace=True)
train_data['hour_bef_ozone'].fillna(value = train_data['hour_bef_ozone'].sum() / (len(train_data) - len(str(train_data['hour_bef_ozone'].isna().sum()))), inplace=True)
train_data['hour_bef_pm10'].fillna(value = train_data['hour_bef_pm10'].sum() / (len(train_data) - len(str(train_data['hour_bef_pm10'].isna().sum()))), inplace=True)
train_data['hour_bef_pm2.5'].fillna(value = train_data['hour_bef_pm2.5'].sum() / (len(train_data) - len(str(train_data['hour_bef_pm2.5'].isna().sum()))), inplace=True)

In [12]:
test_data['hour_bef_temperature'].fillna(value = test_data['hour_bef_temperature'].sum() / (len(test_data) - len(str(test_data['hour_bef_temperature'].isna().sum()))), inplace=True)
test_data['hour_bef_precipitation'].fillna(value = test_data['hour_bef_precipitation'].sum() / (len(test_data) - len(str(test_data['hour_bef_precipitation'].isna().sum()))), inplace=True)
test_data['hour_bef_windspeed'].fillna(value = test_data['hour_bef_windspeed'].sum() / (len(test_data) - len(str(test_data['hour_bef_windspeed'].isna().sum()))), inplace=True)
test_data['hour_bef_humidity'].fillna(value = test_data['hour_bef_humidity'].sum() / (len(test_data) - len(str(test_data['hour_bef_humidity'].isna().sum()))), inplace=True)
test_data['hour_bef_visibility'].fillna(value = test_data['hour_bef_visibility'].sum() / (len(test_data) - len(str(test_data['hour_bef_visibility'].isna().sum()))), inplace=True)
test_data['hour_bef_ozone'].fillna(value = test_data['hour_bef_ozone'].sum() / (len(test_data) - len(str(test_data['hour_bef_ozone'].isna().sum()))), inplace=True)
test_data['hour_bef_pm10'].fillna(value = test_data['hour_bef_pm10'].sum() / (len(test_data) - len(str(test_data['hour_bef_pm10'].isna().sum()))), inplace=True)
test_data['hour_bef_pm2.5'].fillna(value = test_data['hour_bef_pm2.5'].sum() / (len(test_data) - len(str(test_data['hour_bef_pm2.5'].isna().sum()))), inplace=True)

In [13]:
X_train = train_data.drop(['id', 'count'], axis=1)
y_train = train_data['count']
X_test = test_data.drop(['id'], axis=1)

In [14]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)

(1459, 9)
(1459,)
(715, 9)


In [15]:
# train data에 대해 PowerTransformer 적합
pt = PowerTransformer()
pt.fit(X_train)

# train data와 test data를 PowerTransformer로 변환
X_train = pt.transform(X_train)
X_test = pt.transform(X_test)

# **2. Modeling**


In [16]:
from flaml import AutoML

MODEL_TIME_BUDGET = 60*2
MODEL_METRIC = 'rmse'
MODEL_TASK = "regression"

auto_model = AutoML()
params = {
    "time_budget": MODEL_TIME_BUDGET,
    "metric": MODEL_METRIC,
    "task": MODEL_TASK,
    "seed": 43,
    "early_stop": True,
    "estimator_list": ['lgbm', 'rf', 'xgboost'], # 사용할 모델 리스트
}
auto_model.fit(X_train, y_train, **params)

[flaml.automl.logger: 04-29 09:26:57] {1682} INFO - task = regression
[flaml.automl.logger: 04-29 09:26:57] {1689} INFO - Data split method: uniform
[flaml.automl.logger: 04-29 09:26:57] {1692} INFO - Evaluation method: cv
[flaml.automl.logger: 04-29 09:26:57] {1790} INFO - Minimizing error metric: rmse
[flaml.automl.logger: 04-29 09:26:57] {1900} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'xgboost']
[flaml.automl.logger: 04-29 09:26:57] {2210} INFO - iteration 0, current learner lgbm
[flaml.automl.logger: 04-29 09:26:58] {2336} INFO - Estimated sufficient time budget=505s. Estimated necessary time budget=1s.
[flaml.automl.logger: 04-29 09:26:58] {2383} INFO -  at 0.1s,	estimator lgbm's best error=67.6829,	best estimator lgbm's best error=67.6829
[flaml.automl.logger: 04-29 09:26:58] {2210} INFO - iteration 1, current learner lgbm
[flaml.automl.logger: 04-29 09:26:58] {2383} INFO -  at 0.1s,	estimator lgbm's best error=67.6829,	best estimator lgbm's best error=67.6829
[fl

In [17]:
# 가장 좋은 모델의 rmse와 하이퍼파라미터 출력
print(f"Best model: {auto_model.best_config}")
print(f"Best hyperparameters: {auto_model.best_estimator}")
print(f"Best validation score (rmse): {auto_model.best_loss}")
print(f"Feature importances: {auto_model.feature_importances_}")

Best model: {'n_estimators': 284, 'max_leaves': 27, 'min_child_weight': 0.35654827426313257, 'learning_rate': 0.06288219008048522, 'subsample': 0.9037492396879444, 'colsample_bylevel': 1.0, 'colsample_bytree': 0.8377243291604277, 'reg_alpha': 0.002404996678147712, 'reg_lambda': 0.5663199821735411}
Best hyperparameters: xgboost
Best validation score (rmse): 37.33475183934837
Feature importances: [0.4354788  0.14175826 0.16733897 0.04689779 0.04900004 0.03682588
 0.0527533  0.03968558 0.0302615 ]


## AutoML Results

1. Best model: `{'n_estimators': 284, 'max_leaves': 27, 'min_child_weight': 0.35654827426313257, 'learning_rate': 0.06288219008048522, 'subsample': 0.9037492396879444, 'colsample_bylevel': 1.0, 'colsample_bytree': 0.8377243291604277, 'reg_alpha': 0.002404996678147712, 'reg_lambda': 0.5663199821735411}`
2. Best hyperparameters: xgboost
3. Best validation score (rmse): 37.33475183934837

In [20]:
ypred = auto_model.predict(X_test)

# **3. Conclusion**

In [21]:
submission_data['count'] = ypred

In [22]:
submission_data

,id,count
0,0,112.478584
1,1,183.900940
2,2,53.986233
3,4,41.442368
4,5,29.073654
...,...,...
710,2148,53.337219
711,2149,62.314411
712,2165,128.918152
713,2166,129.771347


In [23]:
submission_data.to_csv('model.csv', index=False)